# Offline

In [ ]:
import os
import csv
import time
import requests
import json

In [ ]:
# Load environment variables
gemini_api_key = "GEMINI_API_KEY"  # Changed variable name to reflect Gemini

# Define the Gemini API endpoint
GEMINI_API_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent" # Corrected endpoint

# Define the questions and inputs
questions = [
    {"instruction": "Siapa presiden pertama Indonesia dan sebutkan 3 pencapaiannya", "input_data": ""},
    {"instruction": "Berapa jumlah hari dalam seminggu dan sebutkan nama-namanya dalam bahasa Inggris", "input_data": ""},
    {"instruction": "Tolong jawab operasi matematika ini: (10 + 5) * 2 - 8 / 4", "input_data": ""},
    {"instruction": "Apa ibu kota Amerika Serikat dan sebutkan 3 landmark terkenalnya", "input_data": ""},
    {"instruction": "Apa ibu kota Indonesia dan jelaskan sejarah singkatnya", "input_data": ""},
    {"instruction": "Jelaskan perbedaan antara algoritma merge sort dan quick sort", "input_data": ""},
    {"instruction": "Terjemahkan kalimat ini ke dalam bahasa Prancis: 'Saya sangat senang belajar hal baru hari ini'", "input_data": ""},
    {"instruction": "Buatlah sebuah fungsi Python untuk menghitung faktorial dari sebuah bilangan", "input_data": "5"},
    {"instruction": "Sebutkan 5 planet di tata surya kita dan urutkan berdasarkan jaraknya dari Matahari", "input_data": ""},
    {"instruction": "Jelaskan konsep dasar dari machine learning dan berikan contoh penerapannya", "input_data": ""}
]

# Prepare CSV file
csv_filename = "gemini_answers.csv"  # Changed filename to reflect Gemini

In [ ]:
def get_gemini_response(prompt):
    """
    Sends a prompt to the Gemini API and returns the response.

    Args:
        prompt (str): The prompt to send to the Gemini API.

    Returns:
        str: The text response from the Gemini API, or None on error.
    """
    url = f"{GEMINI_API_ENDPOINT}?key={gemini_api_key}"  # Corrected URL construction
    headers = {
        'Content-Type': 'application/json'
    }
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        response_json = response.json()

        # Print the entire JSON response for debugging
        # print(f"Gemini API Response: {json.dumps(response_json, indent=2)}")

        # Extract the text from the response.  Check for the existence of the keys.
        if 'candidates' in response_json and len(response_json['candidates']) > 0 and \
           'content' in response_json['candidates'][0] and \
           'parts' in response_json['candidates'][0]['content'] and \
           len(response_json['candidates'][0]['content']['parts']) > 0 and \
           'text' in response_json['candidates'][0]['content']['parts'][0]:
            return response_json['candidates'][0]['content']['parts'][0]['text']
        else:
            print("Error: Unexpected response structure from Gemini API.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}. Response text: {response.text}")
        return None
    except KeyError as e:
        print(f"Error: Key not found in response: {e}.  Response: {response.text}")
        return None

# Function to check relevancy using Gemini
def check_relevancy(question, answer):
    if not gemini_api_key:
        raise ValueError("GEMINI_API_KEY is not set in Colab's user secrets. Please set it.")

    url = f"{GEMINI_API_ENDPOINT}?key={gemini_api_key}"
    headers = {"Content-Type": "application/json"}
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"Question: {question} \n Answer: {answer} \n Is the answer relevant and correct? Reply only with 1 or 0."
                    }
                ]
            }
        ]
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        response_json = response.json()

        if 'candidates' in response_json and len(response_json['candidates']) > 0 and \
           'content' in response_json['candidates'][0] and \
           'parts' in response_json['candidates'][0]['content'] and \
           len(response_json['candidates'][0]['content']['parts']) > 0 and \
           'text' in response_json['candidates'][0]['content']['parts'][0]:
            relevancy_str = response_json['candidates'][0]['content']['parts'][0]['text'].strip()
            if relevancy_str in ("0", "1"):
                return int(relevancy_str)
            else:
                print(f"Warning: Gemini returned an invalid relevancy score: {relevancy_str}")
                return 0
        else:
            print(f"Warning: Unexpected response structure from Gemini: {response_json}")
            return 0

    except requests.exceptions.RequestException as e:
        print(f"Error during Gemini request: {e}")
        return 0
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from Gemini: {e}")
        return 0
    except KeyError as e:
        print(f"KeyError: {e}. Response from Gemini: {response.text}")
        return 0

In [ ]:
# Open CSV file and write headers
relevance_scores = []
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["instruction", "input_data", "answer", "relevancy"])

    # Send each question to the Gemini API
    for question in questions:
        prompt = question["instruction"] + (" " + question["input_data"] if question["input_data"] else "")
        answer = get_gemini_response(prompt)

        if answer is not None: # Only proceed if we got a valid answer
            print(f"Question: {question['instruction']}")
            print(f"Answer: {answer}")
            relevancy = check_relevancy(question["instruction"], answer)
            relevance_scores.append(relevancy)
            writer.writerow([question["instruction"], question["input_data"], answer, relevancy])
        else:
            print(f"Question: {question['instruction']}")
            print(f"Answer: {answer}")
            relevancy = 0
            relevance_scores.append(relevancy)
            writer.writerow([question["instruction"], question["input_data"], "Failed to get answer", relevancy])

        time.sleep(1)  # Small delay to avoid rate limiting

# Calculate and display average relevancy
average_relevancy = sum(relevance_scores) / len(relevance_scores) if relevance_scores else 0
print(f"Average relevancy score: {average_relevancy}")
print(f"Results saved to {csv_filename}")

Question: Siapa presiden pertama Indonesia dan sebutkan 3 pencapaiannya
Answer: Presiden pertama Indonesia adalah **Ir. Soekarno**.

Berikut 3 pencapaiannya yang signifikan:

1.  **Memproklamasikan Kemerdekaan Indonesia (17 Agustus 1945):** Bersama dengan Mohammad Hatta, Soekarno memproklamasikan kemerdekaan Indonesia, menandai berakhirnya penjajahan Belanda dan dimulainya era baru bagi bangsa Indonesia. Ini adalah fondasi utama bagi berdirinya negara Indonesia.

2.  **Mempersatukan Bangsa Indonesia:** Soekarno memainkan peran penting dalam mempersatukan berbagai suku, budaya, dan agama di seluruh kepulauan Indonesia di bawah semboyan "Bhinneka Tunggal Ika" (Berbeda-beda tetapi tetap satu). Ia membangun identitas nasional yang kuat.

3.  **Menyelenggarakan Konferensi Asia-Afrika (KAA) 1955:** Soekarno menjadi tokoh kunci dalam penyelenggaraan Konferensi Asia-Afrika di Bandung. KAA menjadi tonggak penting dalam gerakan non-blok dan menyuarakan aspirasi negara-negara berkembang untuk beb

# Online

In [ ]:
# Contoh logika untuk menyimpan feedback pengguna
user_feedback = {"like": 0, "dislike": 0}

def record_feedback(feedback_type):
    if feedback_type == "like":
        user_feedback["like"] += 1
    elif feedback_type == "dislike":
        user_feedback["dislike"] += 1

# Mencatat feedback pengguna
record_feedback("like")
print(user_feedback)

{'like': 1, 'dislike': 0}


In [ ]:
# Contoh logika untuk menghitung klik tombol regenerasi
regenerate_count = 0

def regenerate_response():
    global regenerate_count
    regenerate_count += 1

# Pengguna meminta jawaban ulang
regenerate_response()
regenerate_response()
regenerate_response()
print(f"Jumlah regenerasi: {regenerate_count}")

Jumlah regenerasi: 3
